In [ ]:
acu_conn_select()

In [ ]:
xl_env = xl_init_env(nb_name="AptWeeksMove", 
                     #in_sheet_skip_rows=1, in_sheet_header=0,
                     in_sheet_use_cols=6,
                     db_name=acu_conn_selector.value, 
                    )
xl_env

In [ ]:
acu_curs.callproc('P_PROC_SET', ('DBA_MOVE_SALES_INV', xl_env['in_file_name'], 'T_IMD_Delete_Insert'))

In [ ]:
# delete records from last run
sql = "DELETE FROM T_IMD WHERE IMD_XL_FNAM = :fnam AND IMD_PROCESSED IS NULL"
acu_curs.execute(sql, fnam=xl_env['in_file_name'])
acu_curs.rowcount

In [ ]:
# check if passed row/record (sd) of spreadsheet data is complete and correct
#last_apt_week_completed_date = xl_env['in_records'].iloc[0]['DATE COMPLETED']
def check_col_values(sd, col_list):
    #global last_apt_week_completed_date
    #print(sd, col_list)
    for cn in col_list:
        #print(sd[cn])
        if not sd[cn] or sd[cn] == np.NaN:
            raise ValueError("Missing/Empty value {!r} in column {} of record {}".format(sd[cn], cn, sd))
    #if sd['IMD_COMPLETED'] < last_apt_week_completed_date:
    #    raise ValueError("Sheets and sheet rows have to be ordered ascending by Completion Date")
    #last_apt_week_completed_date = sd['IMD_COMPLETED']
    

# insert in_records into T_IMD
current_year = datetime.datetime.now().year
cols = "IMD_YEAR, IMD_WKREF, IMD_CDREF, IMD_NEW_CDREF, IMD_NEXT_OCCY, IMD_COMPLETED, IMD_XL_FNAM"
coll = cols.split(", ")
rows_inserted = 0
for i, r in xl_env['in_records'].iterrows():
    bd = dict(IMD_YEAR=current_year,
              IMD_WKREF=r['APT/WEEK'],
              IMD_CDREF=r['PREVIOUS OWNER'][:7],  # ensure removing of trailing C char
              IMD_NEW_CDREF=r['NEW OWNER'][:7],
              IMD_NEXT_OCCY=r['OCCUPANCY YEAR'],
              # is already Timestamp: IMD_COMPLETED=datetime.datetime.strptime(r['DATE COMPLETED'], '%Y-%m-%d').date(),
              IMD_COMPLETED=r['DATE COMPLETED'],
              IMD_XL_FNAM=xl_env['in_file_name'],
             )
    check_col_values(bd, coll)
    sql = "INSERT INTO T_IMD (" + cols + ") values (:" + ", :".join(coll) + ")"
    acu_curs.execute(sql, **bd)
    rows_inserted += acu_curs.rowcount
print(rows_inserted)
# .. and complete missing column values
acu_curs.execute("""UPDATE T_IMD set IMD_RSREF = F_RESORT(IMD_WKREF), 
                                     IMD_APREF = substr(IMD_WKREF, 1, instr(IMD_WKREF, '-') - 1),
                                     IMD_WEEK = substr(IMD_WKREF, instr(IMD_WKREF, '-') + 1),
                                     IMD_NEW_ITREF = case when (select count(*) from T_IT where IT_OWREF = IMD_NEW_CDREF) = 1
                                                          then (select IT_CODE from T_IT where IT_OWREF = IMD_NEW_CDREF)
                                                          when IMD_NEW_CDREF = 'A000021'             then 'OWND'
                                                          when IMD_NEW_CDREF = 'A000010'             then 'SPIN'
                                                          when substr(IMD_NEW_CDREF, 1, 1) != 'A'    then 'OWND' 
                                                          end
                    WHERE IMD_XL_FNAM = :fnam AND IMD_PROCESSED IS NULL
                 """, fnam=xl_env['in_file_name'])
acu_conn.commit()
acu_curs.rowcount

In [ ]:
sql = """
select IMD_WKREF as Apt_Wk, IMD_RSREF as Resort
     , IMD_CDREF as Prev_Owner, IMD_NEW_CDREF as Next_Owner
     --, (select WK_OWREF from V_OWNED_WEEKS where DW_WKREF = IMD_WKREF) as Deal_Owner
     , (select WK_OWREF from T_WK where WK_CODE = IMD_WKREF) as Inv_Owner
     , (select OWNERID from T_AOWN where WK_CODE = IMD_WKREF and YEAR = to_char(sysdate, 'YYYY')) as MFee_Owner
     , IMD_YEAR as Move_Year, IMD_NEXT_OCCY as Next_Occy
     , IMD_LOADED
     , substr(''
         || case when length(IMD_APREF) not in (4, case IMD_RSREF when 'PBC' then 3 else -1 end)
                 then ', IMD_APREF has wrong size' end
         || case when length(IMD_WEEK) != case IMD_RSREF when 'CPA' then 4 else 2 end
                 then ', IMD_WEEK has wrong size' end
         || case when length(IMD_WKREF) not in (7, case IMD_RSREF when 'PBC' then 6 when 'CPA' then 9 else -1 end) 
                 then ', IMD_WKREF has wrong size' end
         || case when not exists (select NULL from T_CD where CD_CODE = IMD_CDREF) 
                 then ', IMD_CDREF is invalid' end
         || case when not exists (select NULL from V_OWNED_WEEKS
                                   where DW_WKREF = IMD_WKREF
                                     and (WK_OWREF = IMD_CDREF or substr(WK_OWREF, 1, 1) = 'A' and IMD_CDREF = 'A000500'))
                 then ', Not owned by ' || IMD_CDREF || ' (but by ' || (select WK_OWREF from T_WK where WK_CODE = IMD_WKREF) || ')'
                      || case when (select count(*) from T_IMD b
                                     where b.IMD_WKREF = a.IMD_WKREF 
                                       and b.IMD_XL_FNAM = a.IMD_XL_FNAM and b.IMD_PROCESSED is NULL) > 1
                              then ' Multiple Moves ('  || (select f_stragg(b.IMD_CDREF || '->' || b.IMD_NEW_CDREF || '@' || b.IMD_COMPLETED) from T_IMD b
                                                             where b.IMD_WKREF = a.IMD_WKREF 
                                                               and b.IMD_XL_FNAM = a.IMD_XL_FNAM and b.IMD_PROCESSED is NULL
                                                             -- NOT ALLOWED HERE: order by b.IMD_COMPLETED
                                                            ) 
                                  || ')' || case when exists (select NULL from T_IMD b
                                                               where b.IMD_WKREF = a.IMD_WKREF 
                                                                 and b.IMD_XL_FNAM = a.IMD_XL_FNAM
                                                                 and b.IMD_PROCESSED is NULL
                                                                 and b.IMD_COMPLETED < a.IMD_COMPLETED)
                                                  then ' IGNOREABLE WARNING (there is another movement with earlier completion date)'
                                                  else ' with wrong completion date order' end end end
         || case when not exists (select NULL from T_WK where WK_CODE = IMD_WKREF) 
                 then ', IMD_WKREF is invalid'
                 else case when not exists (select NULL from T_WK, T_DW where WK_CODE = IMD_WKREF and WK_DWREF = DW_CODE) 
                           then ', WK->DW link broken'
                           else case when exists (select NULL from T_WK, T_DW
                                                   where WK_CODE = IMD_WKREF and WK_DWREF = DW_CODE
                                                     and (WK_OCCY > IMD_NEXT_OCCY or DW_OCCY > IMD_NEXT_OCCY)) 
                                     then ', Current/Old occupancy year (' 
                                          || (select greatest(WK_OCCY, DW_OCCY) from T_WK, T_DW
                                               where WK_CODE = IMD_WKREF and WK_DWREF = DW_CODE) 
                                          || ') greater than ' || IMD_NEXT_OCCY || ' (WK=' 
                                          || (select WK_OCCY from T_WK where WK_CODE = IMD_WKREF) || ' DW=' 
                                          || (select DW_OCCY from T_WK, T_DW
                                               where WK_CODE = IMD_WKREF and WK_DWREF = DW_CODE) || ')' 
                                end
                      end
         || case when exists (select NULL from T_CD where CD_CODE = IMD_CDREF and CD_STATUS in (666, 667))
                 then ', Deal cannot be created for blacklisted old owner ' || IMD_CDREF end
         || case when exists (select NULL from T_CD where CD_CODE = IMD_NEW_CDREF and CD_STATUS in (666, 667))
                 then ', Deal cannot be created for blacklisted new owner ' || IMD_NEW_CDREF end
            end
         , 3) as Discrepancies
  from T_IMD a
 where IMD_XL_FNAM = :fnam and IMD_PROCESSED is NULL
   and ( length(IMD_APREF) not in (4, case IMD_RSREF when 'PBC' then 3 else -1 end)
      or length(IMD_WEEK) != case IMD_RSREF when 'CPA' then 4 else 2 end
      or length(IMD_WKREF) not in (7, case IMD_RSREF when 'PBC' then 6 when 'CPA' then 9 else -1 end)
      or not exists (select NULL from T_CD where CD_CODE = IMD_CDREF)
      or not exists (select NULL from V_OWNED_WEEKS
                      where DW_WKREF = IMD_WKREF
                        and (WK_OWREF = IMD_CDREF or substr(WK_OWREF, 1, 1) = 'A' and IMD_CDREF = 'A000500'))
      or not exists (select NULL from T_WK where WK_CODE = IMD_WKREF)
      or not exists (select NULL from T_WK, T_DW where WK_CODE = IMD_WKREF and WK_DWREF = DW_CODE)
      or exists (select NULL from T_WK, T_DW where WK_CODE = IMD_WKREF and WK_DWREF = DW_CODE
                                               and (WK_OCCY > IMD_NEXT_OCCY or DW_OCCY > IMD_NEXT_OCCY))
      or exists (select NULL from T_CD where CD_CODE = IMD_CDREF and CD_STATUS in (666, 667))
      or exists (select NULL from T_CD where CD_CODE = IMD_NEW_CDREF and CD_STATUS in (666, 667))
       )
 order by IMD_WKREF
"""
xl_add_sheet_from_sql(xl_env, sql, "Discrep_1", "1. check for correct apt-week and owner data",
                     bind_vars=dict(fnam=xl_env['in_file_name']))

In [ ]:
sql = """
select WK_CODE as Apt_Wk, WK_OWREF as Acumen_Owner
     , (select OWNERID from T_AOWN where WK_CODE = IMD_WKREF and YEAR = to_char(sysdate, 'YYYY')) as MFee_Owner
     , IMD_CDREF as Prev_Owner, IMD_NEW_CDREF as Next_Owner
     , WK_ITREF as Prev_Inv_Type, IMD_NEW_ITREF as Next_Inv_Type
     , WK_STATUS as Wk_Status
     , F_STATUS(WK_STATUS) as Short_Wk_Status_Desc
     , F_STATUS(WK_STATUS, WK_CODE, LW_CODE, DW_CODE) as Long_Wk_Status_Desc
     , LW_STATUS as List_Status
     , substr(''
        || case when WK_ITREF in ('RE28', 'FI28') then ', Invalid Inventory Type' end
        || case when not exists (select NULL from T_IT where IT_CODE = IMD_NEW_ITREF)
                then ', New inventory type is invalid/empty for ' || IMD_NEW_CDREF end
        || case when WK_STATUS not in (770, 790, 890) then ', Invalid WK Status' 
                else case when substr(F_STATUS(WK_STATUS, WK_CODE, LW_CODE, DW_CODE), 1, 3) not in ('790', '890')
                          then ', Unexpected WK Status' end
           end
        || case when (nvl(LW_STATUS, 0) in (850, 880, 881)) then ', Invalid LW Status' end
        || case when WK_OWREF <> IMD_CDREF and (IMD_CDREF != 'A000500' or substr(WK_OWREF, 1, 1) != 'A')
                then ', Different Owner Ref' end
        || case when substr(F_STATUS(WK_STATUS, WK_CODE, LW_CODE, DW_CODE), 1, 3) = '770' or WK_STATUS = 770
                then ', Financed' end
        , 3) as Discrepancies
  from T_IMD a, T_DW, T_WK left outer join T_LW on WK_LWREF = LW_CODE
 where IMD_XL_FNAM = :fnam and IMD_PROCESSED is NULL
   and IMD_WKREF = WK_CODE
   and WK_DWREF = DW_CODE
   and (WK_ITREF in ('RE28', 'FI28') 
     or not exists (select NULL from T_IT where IT_CODE = IMD_NEW_ITREF)
     or WK_STATUS not in (770, 790, 890)
     or substr(F_STATUS(WK_STATUS, WK_CODE, LW_CODE, DW_CODE), 1, 3) not in ('790', '890')
     or (nvl(LW_STATUS, 0) in (850, 880, 881))
     or WK_OWREF <> IMD_CDREF and (IMD_CDREF != 'A000500' or substr(WK_OWREF, 1, 1) != 'A')
        and not exists (select NULL from T_IMD b where b.IMD_WKREF = a.IMD_WKREF and b.IMD_PROCESSED is NULL
                                                   and (b.IMD_NEW_CDREF = a.IMD_CDREF or b.IMD_COMPLETED < a.IMD_COMPLETED))
     )
 order by WK_CODE
"""
xl_add_sheet_from_sql(xl_env, sql, "Discrep_2", 
                      "2. apt-weeks with either invalid week status or inventory type (empty)/RE28/FI28 or LW status 850,880,881",
                     bind_vars=dict(fnam=xl_env['in_file_name']))

In [ ]:
sql = """
select WK_CODE as Apt_Wk, WK_CEDE_STATUS as Cede_Status, WK_OWREF as Acumen_Owner
     , WK_POINTS as Wk_Points
     , (select case when DW_POINTS > 0 then 'i' || DW_POINTS end from T_DW where DW_CODE = WK_DWREF) as Ind_Float_Points
     , (select f_stragg(DAC_DLREF) from T_DW, T_DAC where DW_CODE = WK_DWREF and DW_DLREF = DAC_DLREF
                                                      and WK_APREF = DAC_APREF and WK_WEEK = DAC_WEEK) as Ceding_Deals
     , (select decode(WK_CEDE_STATUS, 2,'Y ', 1,'y', '') || decode(DAC_CODE, NULL,'', 'S') from T_DW, T_DAC
         where DW_CODE = WK_DWREF and DW_DLREF = DAC_DLREF and WK_APREF = DAC_APREF and WK_WEEK = DAC_WEEK) as Ceded_Flags
  from T_IMD, T_WK
 where IMD_XL_FNAM = :fnam and IMD_PROCESSED is NULL
   and IMD_WKREF = WK_CODE
   and (WK_CEDE_STATUS <> 0
     or exists (select NULL from T_DW, T_DAC where DW_CODE = WK_DWREF and DW_DLREF = DAC_DLREF
                                               and WK_APREF = DAC_APREF and WK_WEEK = DAC_WEEK)
       )
 order by WK_CODE
"""
xl_add_sheet_from_sql(xl_env, sql, "Discrep_3a", 
                      "3a. suspended apt-weeks with RCI Points ceded/in-ceding flag or associated ceding deal",
                     bind_vars=dict(fnam=xl_env['in_file_name']))

In [ ]:
sql = """
select IMD_CDREF as Owner
     , listagg(WK_CODE, ',') within group (order by WK_CODE) as IN_AptWeeks,        OUT_AptWeeks
     , listagg(WK_POINTS, '+') within group (order by WK_CODE) as IN_RciPoints,     OUT_RciPoints
     , sum(WK_POINTS) as IN_TotalRciPoints,                                         OUT_TotalRciPoints
     , listagg(DW_POINTS, '+') within group (order by DW_WKREF) as IN_IndiPoints,   OUT_IndiPoints
     , sum(DW_POINTS) as IN_TotalIndiPoints,                                        OUT_TotalIndiPoints
     , sum(AP_OCCUPANTS) as IN_TotalOccupants,                                      OUT_TotalOccupants
     , listagg(AT_GENERIC, ',') within group (order by AT_GENERIC) as IN_AptSizes,  OUT_AptSizes
  from T_IMD ii, T_WK iw, T_DW id, T_AP ia, T_AT it
     , (select oi.IMD_NEW_CDREF
             , listagg(ow.WK_CODE, ',') within group (order by ow.WK_CODE)       as OUT_AptWeeks
             , listagg(ow.WK_POINTS, '+') within group (order by ow.WK_CODE)     as OUT_RciPoints
             , sum(ow.WK_POINTS)                                                 as OUT_TotalRciPoints
             , listagg(od.DW_POINTS, '+') within group (order by od.DW_WKREF)    as OUT_IndiPoints
             , sum(od.DW_POINTS)                                                 as OUT_TotalIndiPoints
             , sum(oa.AP_OCCUPANTS)                                              as OUT_TotalOccupants
             , listagg(ot.AT_GENERIC, ',') within group (order by ot.AT_GENERIC) as OUT_AptSizes
          from T_IMD oi, T_WK ow, T_DW od, T_AP oa, T_AT ot
         where oi.IMD_WKREF = ow.WK_CODE
           and ow.WK_DWREF = od.DW_CODE
           and ow.WK_APREF = oa.AP_CODE
           and oa.AP_ATREF = ot.AT_CODE
           and oi.IMD_XL_FNAM = :fnam and oi.IMD_PROCESSED is NULL
           --not needed: and oi.IMD_NEW_CDREF is not NULL and substr(oi.IMD_NEW_CDREF, 1, 1) <> 'A'
         group by oi.IMD_NEW_CDREF
        ) oi
 where ii.IMD_WKREF = iw.WK_CODE
   and iw.WK_DWREF = id.DW_CODE
   and iw.WK_APREF = ia.AP_CODE
   and ia.AP_ATREF = it.AT_CODE
   and ii.IMD_CDREF = oi.IMD_NEW_CDREF
   and ii.IMD_XL_FNAM = :fnam and ii.IMD_PROCESSED is NULL
   and ii.IMD_NEW_CDREF is not NULL and substr(ii.IMD_CDREF, 1, 1) <> 'A'
 group by ii.IMD_CDREF, OUT_AptWeeks, OUT_RciPoints, OUT_TotalRciPoints, OUT_IndiPoints, OUT_TotalIndiPoints, OUT_TotalOccupants, OUT_AptSizes
 -- uncomment next line for to show only the ones having 
 --having sum(WK_POINTS) / OUT_TotalRciPoints not between 0.667 and 1.5
 --having sum(WK_POINTS) / OUT_TotalRciPoints not between 0.5 and 2.0
 order by ii.IMD_CDREF
"""
# SISO: unfortunately one-to-one mapping is NOT provided by XL/Rebeccas spreadsheet, so we can only compare exchange summaries
xl_add_sheet_from_sql(xl_env, sql, "Discrep_3b", 
                      "3-B. owner movement/exchange balance: client ref, apt-weeks, old/new RCI points, old/new RCI total points, old/new Individual points, old/new Individual total points, old/new total occupants, old/new unit sizes",
                     bind_vars=dict(fnam=xl_env['in_file_name']))

In [ ]:
sql = """
select WK_CODE as Apt_Wk
     , (select F_STRAGG(IMD_NEW_CDREF) from T_IMD where IMD_WKREF = WK_CODE and IMD_XL_FNAM = :fnam and IMD_PROCESSED is NULL)
       as Next_Owner
     , WK_OWREF as Prev_Owner, WK_OLDOW as Old_Owner
     , (select F_STRAGG(IMD_NEW_ITREF) from T_IMD where IMD_WKREF = WK_CODE and IMD_XL_FNAM = :fnam and IMD_PROCESSED is NULL)
       as Next_Inv_Type
     , WK_ITREF as Prev_Inv_Type
     , (select F_STRAGG(IMD_NEXT_OCCY) from T_IMD where IMD_WKREF = WK_CODE and IMD_XL_FNAM = :fnam and IMD_PROCESSED is NULL)
       as Next_OccY
     , WK_OCCY as Prev_OccY, WK_OOCCY as Old_Occy
     , WK_STATUS as Wk_Status
 from T_WK
 where exists (select NULL from T_IMD where IMD_WKREF = WK_CODE and IMD_XL_FNAM = :fnam and IMD_PROCESSED is NULL)
 order by WK_CODE
"""
xl_add_sheet_from_sql(xl_env, sql, "Log_4", 
                      "4. log for all the moved apt-weeks: WK_CODE, owner, old owner, the inventory type/WK_ITREF and their old and current occupancy year",
                     bind_vars=dict(fnam=xl_env['in_file_name']))

In [ ]:
OLD__sql = """
DECLARE
  lcLogText     varchar2(2000);
  lcLogTxt2     varchar2(2000);
  lcResort      T_RS.RS_CODE%type;
  lnStackLength integer;
  -- fetch all cancelled/suspended weeks/memberships that not were processed before (IMD_PROCESSED is NULL)
  cursor cMFeeSusp is
    select * from T_IMD
     where IMD_PROCESSED is NULL        -- hide excluded and already processed aptWks
       and IMD_XL_FNAM = :fnam
     order by IMD_WKREF, IMD_COMPLETED;
BEGIN
  lnStackLength := F_PROC_STACK_OFF(0);    -- reset procedure stack from previous failed runs
  for rMFeeSusp in cMFeeSusp loop
    BEGIN
      lcLogText := NULL;
      lcLogTxt2 := NULL;
      lcResort := substr(F_RESORT(rMFeeSusp.IMD_APREF), 1, 3);
      P_CREATE_INV_MOVE_DEAL( lcLogText,
                              rMFeeSusp.IMD_WKREF, 
                              rMFeeSusp.IMD_NEW_ITREF,
                              rMFeeSusp.IMD_CDREF,       -- current owner 
                              rMFeeSusp.IMD_YEAR,
                              rMFeeSusp.IMD_NEXT_OCCY,   -- occy for resales/nextOwner
                              False,                     -- NO commit
                              'OWNER_CHANGE',            -- deal type, can only be either CNFS, OWNER_CHANGE, TRANSFER for to prevent error message in P_DEAL_WIZ()->SetDealOwner() (line 930 in P_DEAL_WIZ_V46())
                              790,                       -- WK_STATUS get set always to Completed because these apt-weeks will be used as investments and there are no resales anymore
                              rMFeeSusp.IMD_NEW_CDREF,   -- if NULL then determine by inventory type
                              rMFeeSusp.IMD_COMPLETED    -- def=sysdate
                             );  
      if 1=0       and substr(lcLogText, 1, 4) = 'IMD:' and lcResort = 'CPA' then   
      -- de-activated this extra step first for CPA 2014 batch/list (also kept so for CPA 2015 CXL list)
        -- move from A000101 to IT_OWREF value of passed inventory type, 2015 occy for resales/nextOwner
        P_CREATE_INV_MOVE_DEAL( lcLogTxt2,
                                rMFeeSusp.IMD_WKREF, 
                                case when substr(rMFeeSusp.IMD_APREF, -1) = 'P' then 'CPLT' else 'CPAR' end,
                                'A000101',    
                                rMFeeSusp.IMD_YEAR,
                                rMFeeSusp.IMD_NEXT_OCCY, 
                                False,
                                'TRANSFER'
                               );
        lcLogText := substr(lcLogText || ' / ' || lcLogTxt2, 1, 1998);
      end if;
    EXCEPTION when others then
      lcLogText := substr('ERE: ' || SQLCODE || ':' || substr(SQLERRM, 1, 180) || ', FirstLog=' || lcLogText 
                          || chr(13) || DBMS_UTILITY.FORMAT_ERROR_STACK(), 1, 1998);
    END;
    update T_IMD
       set IMD_LOG_MSG = substr(lcLogText, 1, 1999)
         , IMD_PROCESSED = sysdate
     where IMD_PROCESSED is NULL
       and IMD_XL_FNAM = :fnam
       and IMD_YEAR = rMFeeSusp.IMD_YEAR 
       and IMD_CDREF = rMFeeSusp.IMD_CDREF
       and IMD_NEW_CDREF = rMFeeSusp.IMD_NEW_CDREF
       and IMD_APREF = rMFeeSusp.IMD_APREF
       and IMD_WEEK = rMFeeSusp.IMD_WEEK
       and IMD_WKREF = rMFeeSusp.IMD_WKREF
       and IMD_LOADED = rMFeeSusp.IMD_LOADED;
  end loop;
  -- having COMMIT inside the loop/after UPDATE T_IMD will skip duplicate records (cursor content get updated)
  commit;
END;
"""
processed = list()
bind_vars = dict(fnam=xl_env['in_file_name'])
sql = """
    select IMD_WKREF, IMD_CDREF, IMD_NEW_CDREF, IMD_NEW_ITREF, IMD_COMPLETED, IMD_YEAR, IMD_NEXT_OCCY
      from T_IMD
     where IMD_PROCESSED is NULL        -- hide excluded and already processed aptWks
       and IMD_XL_FNAM = :fnam
     order by IMD_WKREF, IMD_COMPLETED
"""
to_process = pd.read_sql_query(sql, con=acu_conn, params=bind_vars).to_dict(orient='records')
for idx, row in enumerate(to_process):
    ref_param = prepare_ref_param("")
    params = (ref_param,
              row['IMD_WKREF'], 
              row['IMD_NEW_ITREF'],
              row['IMD_CDREF'],       # current owner 
              row['IMD_YEAR'],
              row['IMD_NEXT_OCCY'],   # occy for resales/nextOwner
              False,                  # NO commit
              'OWNER_CHANGE',         # deal type, can only be either CNFS, OWNER_CHANGE, TRANSFER for to prevent error message in P_DEAL_WIZ()->SetDealOwner() (line 930 in P_DEAL_WIZ_V46())
              790,                    # WK_STATUS get set always to Completed because these apt-weeks will be used as investments and there are no resales anymore
              row['IMD_NEW_CDREF'],   # if NULL then determine by inventory type
              row['IMD_COMPLETED'],   # def=sysdate
             )
    print("{}/{} Params".format(idx + 1, len(to_process)), params)
    try:
        acu_curs.callproc('P_CREATE_INV_MOVE_DEAL', params)
        row['IMD_LOG_MSG'] = ref_param.getvalue()
    except Exception as ex:
        row['IMD_LOG_MSG'] = "ERE: exception {} processing data {}".format(ex, row)
    
    bind_vars.update(row)
    sql = """
    update T_IMD
       set IMD_LOG_MSG = substr(:IMD_LOG_MSG, 1, 1999)
         , IMD_PROCESSED = sysdate
     where IMD_PROCESSED is NULL and IMD_LOG_MSG is NULL
       and IMD_WKREF = :IMD_WKREF
       and IMD_CDREF = :IMD_CDREF
       and IMD_NEW_CDREF = :IMD_NEW_CDREF
       and IMD_NEW_ITREF = :IMD_NEW_ITREF
       and IMD_COMPLETED = :IMD_COMPLETED
       and IMD_YEAR = :IMD_YEAR 
       and IMD_NEXT_OCCY = :IMD_NEXT_OCCY
       and IMD_XL_FNAM = :fnam
    """
    #print("Bindvars", bind_vars)
    try:
        acu_curs.execute(sql, **bind_vars)
        acu_conn.commit()
        row['IMD_LOG_MSG'] += "; logged={}".format(acu_curs.rowcount)
    except Exception as ex:
        print("***  EXCEPTION  ***", ex)
        acu_conn.rollback()
        row['IMD_LOG_MSG'] += "; log exception={}".format(ex)
    #print("Row", row)
    processed.append(row)
xl_add_sheet_from_df(xl_env['xl_writer'], pd.DataFrame.from_records(processed), "Log_5", "5. log deal creation")

In [ ]:
sql = """
select WK_CODE as Apt_Wk, WK_OWREF as Next_Owner, WK_OLDOW as Prev_Owner
     , WK_ITREF as Next_Inv_Type
     , WK_OCCY as Next_Occy, WK_OOCCY as Prev_Occy, WK_STATUS as Wk_Status
 from T_WK
 where exists (select NULL from T_IMD where IMD_WKREF = WK_CODE 
                                        --and IMD_PROCESSED is not NULL
                                        and IMD_XL_FNAM = :fnam)
 order by WK_CODE
"""
xl_add_sheet_from_sql(xl_env, sql, "Log_6", "6. Final changes on T_WK columns done by this batch",
                      bind_vars=dict(fnam=xl_env['in_file_name']))

In [ ]:
sql = """
select nvl(IMD_LOG_MSG, 'MOVED ' || IMD_WKREF
                        || ' from ' || IMD_CDREF 
                        || ' to ' || IMD_NEW_CDREF 
                        || ' with next InvType/Occy ' || IMD_NEW_ITREF || '/' || IMD_NEXT_OCCY
                        || ' (new T_WK data (Owner/Inv_Type/OccY): ' 
                        || (select WK_OWREF || '/' || WK_ITREF || '/' || WK_OCCY
                             from T_WK where WK_CODE = IMD_WKREF) || ')'
           ) as LOG_MESSAGE
  from T_IMD
 where IMD_XL_FNAM = :fnam
 order by IMD_LOG_MSG, IMD_WKREF, IMD_COMPLETED
"""
xl_add_sheet_from_sql(xl_env, sql, "Log_7", 
                      "7. protocol of PROCESSED apt-weeks - checking suspension deal creation errors (ERR and ERE are ordered onto the front of this result list)", 
                      bind_vars=dict(fnam=xl_env['in_file_name']))

In [ ]:
sql = """
select IMD_WKREF as Apt_Wk, YEAR as Res_Inv_Year, AOWNL_PRIMARY as Res_Inv_Chg_ID
     , CLUBID as MFee_Resort, AOWN_RSREF as Acumen_Resort
     , AOWN_ROREF as Res_Inv_Type, IMD_YEAR as Move_Year, IMD_NEXT_OCCY as Next_Occy
     , nvl(AOWNL_CHANGES, '(none)') as Res_Inv_Changes
     , substr(''
        || case when substr(AOWNL_CHANGES, 1, 10) <> 'AOWN_ROREF'
                then ', Expected Reservation Inventory Type Change did not happen' end
        || case when substr(AOWNL_CHANGES, 13, 2) <> 'SU'
                then ', Unexpected old Reservation Inventory Type ' || substr(AOWNL_CHANGES, 13, 2) || ' (expected SU)' end 
        || case when substr(AOWNL_CHANGES, 19, 2) not in ('SP', 'HU')
                then ', Unexpected new Reservation Inventory Type ' || substr(AOWNL_CHANGES, 19, 2) || ' (expected SP or HU)' end
        || case when AOWNL_CHANGES is NULL 
                     and not exists (select NULL from T_LU
                                      where LU_CLASS = 'USAGE_RULES' and LU_ID = 'PROV_BOOKING_INV_CHG_LOCK'
                                        and instr(LU_CHAR, AOWN_ROREF) > 0) 
                     and IMD_NEXT_OCCY <= YEAR then ', Missing Reservation Inventory Type Change' end
        , 3) as Discrepancies 
  from T_IMD, T_AOWN, T_AOWNL
 where IMD_XL_FNAM = :fnam and IMD_WKREF = WK_CODE and IMD_YEAR <= YEAR
   and WK_CODE = substr(AOWNL_PRIMARY(+), 1, instr(AOWNL_PRIMARY(+), '~') - 1)
   and YEAR = substr(AOWNL_PRIMARY(+), instr(AOWNL_PRIMARY(+), '~') + 1) 
 order by IMD_WKREF, YEAR
"""
xl_add_sheet_from_sql(xl_env, sql, "Log_8", 
                      "8. reservation inventory change protocol and discrepancies", 
                      bind_vars=dict(fnam=xl_env['in_file_name']))

In [ ]:
acu_curs.callproc('P_PROC_SET', (None, None, None))
xl_exit_env(xl_env)